IMT 2210 - Algebra Lineal para Ciencia de Datos <br>
**Pontificia Universidad Católica de Chile**<br>
**Instituto de Ingeniería Matemática y Computacional**<br>
**Semestre 2023-2**<br>
**Profesor:** Cristobal Rojas <br>

# <h1><center>Proyecto Computacional</center></h1>
# <h3><center>Mateo Andrade B.</center></h3>


In [2]:
import math
import pandas as pd
import numpy as np


## Pregunta 1:
### Clase Vec:

In [3]:
class Vec:
    def __init__ (self, D=[], f={}):
        self.D = D
        self.f = f
    def getitem(self, k):
        if k in self.D:
            if k in self.f:
                return self.f[k]
            else:
                return 0
        else:
            return "Dominio diferente"
    def setitem(self, k, val=0):
        if k not in self.D:
            self.D.append(k)
        #Recordar que en la clase Vec esta indicado que si el valor es 0, no lo agregamos a coordenadas(f):
        if val != 0:
            self.f[k] = val
    def equal(self, u):
        if self.D == u.D:
            for d in self.D:
                if self.getitem(d) != u.getitem(d):
                    return False
            return True
        else:
            return False
    def add(self, u):
        if self.D != u.D:
            return "Dominios diferentes"
        vector_final = Vec(self.D, {})
        for d in self.D:
            vector_final.setitem(d, self.getitem(d) + u.getitem(d))
        return vector_final
    def dot(self, u):
        if self.D != u.D:
            return "Dominios diferentes"
        producto_punto = 0
        for d in self.D:
            producto_punto += self.getitem(d) * u.getitem(d)
        return producto_punto
    def scalar_mul(self, alpha):       
        vector_final = Vec(self.D, {})
        for d in self.D:
            vector_final.setitem(d, self.getitem(d) * alpha)
        return vector_final
    def neg(self):
        vector_final = Vec(self.D, {})
        for d in self.D:
            vector_final.setitem(d, self.getitem(d) * -1)
        return vector_final


## Sistema Recomendador de Películas:

In [4]:
#Leemos los archivos:
movies_csv=[i.strip().split(',') for i in list(open('movies.csv'))]
ratings_csv=[i.strip().split(',') for i in list(open('ratings.csv'))]
ratings_csv.pop(0)
movies_csv.pop(0)

['movie_id', 'movie_name']

#### Paso a)

In [5]:
users = dict()
#Iteramos cada rating:
for rating in ratings_csv:
    #Si el usuario no existe lo creamos:
    if users.get(int(rating[0])) == None:
        #En el pdf se nos pide iterar todas las peliculas para crear el vector, asi que itere todas las peliculas adentro del dominio del vector tal como nos lo pedian:
        users[int(rating[0])] = Vec([int(movie[0]) for movie in movies_csv], {int(rating[1]): int(rating[2])})
    else:
        #aplicamos set_item para agregar el rating a la pelicula usando su movie_id:
        users[int(rating[0])].setitem(int(rating[1]), int(rating[2]))
        

#### Paso b)

In [6]:
movies = dict()
for i in range(1, len(movies_csv) + 1):
    dict_ratings = {}
    for w in range(1, len(users) + 1):
        if users[w].getitem(i) == 0:
            continue
        else:
            dict_ratings[w] = users[w].getitem(i)
    movies[i] = Vec([int(user) for user in users.keys()], dict_ratings)


### Paso c) Definimos la funcion vecinos(users, user_id, k)

In [7]:
def vecinos(users, user_id, k):
    usuario = users[user_id]
    vecinos = {}
    for user in users.keys():
        if user == user_id:
            continue
        else:
            vecinos[user] = usuario.dot(users[user]) / (math.sqrt(usuario.dot(usuario)) * math.sqrt(users[user].dot(users[user])))
    vecinos_sorted = sorted(vecinos.items(), key=lambda x: x[1], reverse=True)
    return vecinos_sorted[:k]

In [8]:
vecinos_uno = vecinos(users, 10, 5)
vecinos_uno

[(308, 0.44838784051796704),
 (840, 0.44506023423521296),
 (666, 0.4441385545838411),
 (561, 0.4432200396511484),
 (716, 0.43406963732938975)]

### Paso d)

In [9]:
def recomendacion(user_id):
    #nos basaremos en las recomendaciones de los 5 vecinos mas cercanos:
    vecinos_usuario = vecinos(users, user_id, 5)
    #lista de los ids de las peliculas que no ha visto, osea que su calificacion sea 0:
    peliculas_novistas = []
    for movie in movies.keys():
        if users[user_id].getitem(movie) == 0:
            peliculas_novistas.append(movie)
    #Ahora sumaremos las estrellas totales de los vecinos por cada pelicula.
    #Es decir si estamos hablando de la pelicula con id = 1 y tenemos dos vecinos y cada vecino la califico con 5 estrellas esa pelicula tendra una suma de 10 estrellas:
    estrellas_totales = {}
    for movie in peliculas_novistas:
        estrellas_totales[movie] = 0
        for vecino in vecinos_usuario:
            estrellas_totales[movie] += users[vecino[0]].getitem(movie)
    estrellas_totales_sorted = sorted(estrellas_totales.items(), key=lambda x: x[1], reverse=True)
    return estrellas_totales_sorted[:10]



In [10]:
recomendacion(10)

[(480, 22),
 (511, 22),
 (514, 22),
 (91, 21),
 (515, 21),
 (50, 20),
 (174, 20),
 (443, 20),
 (520, 20),
 (81, 19)]

# <h2><center>Resolucion de sistemas lineales
</center></h2>

a) Cargar los datos de Costos prod.csv y Recursos.csv a un dataFrame de la librer´ıa pandas e imprimir
estos dataframes en la pantalla, con el fin de entender c´omo se organizan los datos (para la impresi´on
en pantalla se recomienda usar el comando dataFrame.head()).

In [11]:
import pandas as pd


df_costos = pd.read_csv('Costos_producción.csv')
df_recursos = pd.read_csv("Recursos.csv")
df_ordenado = df_costos.sort_values(by='Producto', ascending=True)
df_ordenado
#Tendremos los productos y sus costos de produccion de manera ordenada para seguir los ordenes respectivos de los indices al momento de resolver el sistema lineal

,Unnamed: 0,Producto,Costo de producción
14,14,1,1752625
18,18,2,1719304
27,27,3,1837615
2,2,4,1764282
38,38,5,1742108
15,15,6,1792307
13,13,7,1759963
19,19,8,1780502
43,43,9,1808398
3,3,10,1695162


In [12]:
df_recursos
df_recursos = df_recursos.sort_values(by=['Producto', 'Tipo de recurso'], ascending=[True, True])
df_recursos

,Unnamed: 0,Tipo de recurso,Cantidad,Producto
2079,2079,1,60,1
1302,1302,2,54,1
1389,1389,3,76,1
1689,1689,4,53,1
1647,1647,5,54,1
...,...,...,...,...
510,510,46,40,50
2247,2247,47,71,50
2252,2252,48,58,50
305,305,49,78,50


b) Crear una matriz A utilizando la librer´ıa numpy, donde la entrada i, j de la matriz corresponda a la
cantidad necesaria de la materia prima j para producir el producto i

In [13]:

num_materias_primas = df_recursos['Tipo de recurso'].nunique()
num_productos = df_recursos['Producto'].nunique()


A = np.zeros((num_productos, num_materias_primas))

for index, row in df_recursos.iterrows():
    i = row['Producto'] - 1 
    j = row['Tipo de recurso'] - 1  # 
    A[i, j] = row['Cantidad'] #establecemos la cantidad necesaria de la mateira prima j para producir el producto i


In [14]:
pd_A = pd.DataFrame(A)
pd_A

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,60.0,54.0,76.0,53.0,54.0,71.0,63.0,72.0,53.0,65.0,...,70.0,43.0,44.0,40.0,75.0,49.0,64.0,51.0,70.0,73.0
1,41.0,60.0,45.0,41.0,46.0,65.0,52.0,55.0,64.0,67.0,...,58.0,42.0,54.0,79.0,59.0,58.0,78.0,50.0,75.0,45.0
2,55.0,50.0,72.0,51.0,50.0,56.0,63.0,79.0,60.0,64.0,...,72.0,70.0,41.0,64.0,74.0,66.0,66.0,64.0,72.0,78.0
3,41.0,63.0,65.0,47.0,44.0,61.0,46.0,75.0,65.0,73.0,...,75.0,62.0,45.0,48.0,53.0,47.0,45.0,62.0,57.0,50.0
4,52.0,61.0,61.0,75.0,53.0,63.0,71.0,74.0,50.0,59.0,...,70.0,71.0,70.0,53.0,73.0,71.0,49.0,48.0,54.0,46.0
5,52.0,53.0,43.0,51.0,66.0,61.0,78.0,79.0,46.0,71.0,...,52.0,57.0,40.0,70.0,44.0,66.0,62.0,77.0,61.0,77.0
6,45.0,74.0,44.0,76.0,55.0,43.0,48.0,62.0,75.0,58.0,...,59.0,77.0,47.0,72.0,46.0,57.0,70.0,64.0,58.0,64.0
7,63.0,77.0,67.0,79.0,42.0,68.0,72.0,40.0,56.0,45.0,...,41.0,67.0,66.0,56.0,77.0,58.0,47.0,60.0,73.0,40.0
8,78.0,60.0,77.0,71.0,67.0,61.0,78.0,73.0,58.0,55.0,...,57.0,68.0,60.0,58.0,66.0,44.0,59.0,71.0,50.0,59.0
9,68.0,61.0,73.0,73.0,56.0,70.0,45.0,41.0,74.0,42.0,...,45.0,51.0,74.0,44.0,68.0,59.0,48.0,56.0,51.0,49.0


De esta manera tenemos una matriz 50x50, donde las filas son los productos y cada columna es un diferente item de producción

c) Crear un vector de costos de producci´on con la librer´ıa numpy.

Creamos el vector costos de produccion con indice de producto de 1 a 50, de esta manera se asocia con las filas respectivas de cada producto de la matriz 50x50

In [15]:
costos_produccion = df_ordenado['Costo de producción'].to_numpy()
costos_produccion

array([1752625, 1719304, 1837615, 1764282, 1742108, 1792307, 1759963,
       1780502, 1808398, 1695162, 1779912, 1778551, 1635176, 1793300,
       1832280, 1831478, 1777934, 1676480, 1817541, 1653958, 1853507,
       1659497, 1697236, 1700697, 1739332, 1828149, 1804233, 1698686,
       1830959, 1694014, 1826130, 1771014, 1779345, 1729440, 1812504,
       1816633, 1746407, 1802411, 1832598, 1778015, 1764231, 1777600,
       1788152, 1875284, 1743951, 1833116, 1782662, 1801560, 1828692,
       1706024], dtype=int64)

d) Obtener el precio de cada materia prima resolviendo la ecuaci´on Ax = b, donde A corresponde a la
matriz creada en el item b), b corresponde al vector creado en el item c), y x corresponder´a al vector
de precios buscado (para esto se recomienda usar el m´etodo solve() de la librer´ıa numpy.linalg, que
pueden revisar en: https://numpy.org/doc/stable/reference/generated/numpy.linalg.solve.html)


In [16]:
vector_costos = np.linalg.solve(A, costos_produccion)
dataframe_precio_materia_prima = pd.DataFrame(vector_costos)

dataframe_precio_materia_prima

,0
0,272.0
1,895.0
2,722.0
3,329.0
4,142.0
5,900.0
6,191.0
7,819.0
8,314.0
9,744.0


Resolviendo este sistema lineal hemos obtenidos las 50 diferentes incognitas que se requieren para producir los 50 productos diferentes. 

P = Producto ,  I = Item
|      | Col1 | Col2 | Col3 | Col4 | ... | Col50 |
|------|------|------|------|------|-----|-------|
| Row1 | P1I1 | P1I2 | P1I3 | P1I4 | ... | P1I50 |
| Row2 | P2I1 | P2I2 | P2I3 | P2I4 | ... | P2I50 |
| Row3 | P3I1 | P3I2 | P3I3 | P3I4 | ... | P3I50 |
| ...  | ...  | ...  | ...  | ...  | ... | ...   |
| Row50|P50I1 |P50I2 |P50I3 |P50I4 | ... |P50I50 |



e) Cargar los datos de Recursos new.csv y crear una matriz de numpy tal como hizo en el inciso b), es
decir, una matriz donde la entrada i, j de la matriz corresponda a la cantidad necesaria de la materia
prima j para producir el producto i.

In [22]:
df_recursos_new = pd.read_csv("Recursos_new.csv")
df_recursos_new

,Unnamed: 0,Tipo de recurso,Cantidad,Producto
0,0,1,58,1
1,1,2,42,1
2,2,3,99,1
3,3,4,75,1
4,4,5,45,1
...,...,...,...,...
2495,2495,46,9,50
2496,2496,47,3,50
2497,2497,48,45,50
2498,2498,49,87,50


In [20]:

num_materias_primas = df_recursos_new['Tipo de recurso'].nunique()
num_productos = df_recursos_new['Producto'].nunique()
A_new = np.zeros((num_productos, num_materias_primas))
for index, row in df_recursos_new.iterrows():
    i = row['Producto'] - 1 
    j = row['Tipo de recurso'] - 1 
    A_new[i, j] = row['Cantidad']
pd_A_new = pd.DataFrame(A_new)
pd_A_new

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,58.0,42.0,99.0,75.0,45.0,27.0,29.0,24.0,33.0,68.0,...,20.0,66.0,87.0,71.0,32.0,24.0,9.0,58.0,24.0,19.0
1,22.0,2.0,1.0,31.0,52.0,74.0,3.0,33.0,26.0,12.0,...,3.0,6.0,47.0,14.0,66.0,30.0,52.0,6.0,65.0,77.0
2,49.0,64.0,89.0,95.0,81.0,43.0,55.0,95.0,48.0,93.0,...,98.0,54.0,44.0,0.0,21.0,53.0,23.0,4.0,88.0,13.0
3,66.0,15.0,75.0,15.0,98.0,52.0,70.0,82.0,25.0,69.0,...,22.0,60.0,98.0,86.0,31.0,56.0,10.0,76.0,43.0,29.0
4,99.0,13.0,15.0,17.0,7.0,53.0,8.0,67.0,78.0,52.0,...,43.0,8.0,9.0,43.0,61.0,92.0,71.0,95.0,32.0,7.0
5,65.0,62.0,42.0,45.0,36.0,7.0,62.0,38.0,57.0,67.0,...,21.0,68.0,44.0,93.0,54.0,55.0,68.0,33.0,76.0,36.0
6,29.0,78.0,62.0,75.0,33.0,88.0,59.0,36.0,99.0,30.0,...,65.0,41.0,99.0,57.0,4.0,61.0,6.0,51.0,92.0,28.0
7,94.0,28.0,52.0,11.0,77.0,60.0,37.0,11.0,22.0,14.0,...,66.0,83.0,51.0,38.0,1.0,7.0,91.0,0.0,33.0,11.0
8,18.0,49.0,52.0,19.0,7.0,38.0,99.0,62.0,49.0,70.0,...,70.0,59.0,27.0,38.0,3.0,79.0,93.0,99.0,21.0,1.0
9,32.0,45.0,92.0,81.0,28.0,0.0,14.0,78.0,27.0,70.0,...,97.0,1.0,33.0,22.0,10.0,66.0,6.0,23.0,26.0,65.0


f) Calcular e imprimir en pantalla el producto matriz vector A · x donde A es la matriz que arm´o en el
inciso e), y x es el vector de costos que obtuvo en d). Tenga cuidado de ordenar bien el vector de costos
si es necesario.

Realizamos el producto matrix Vector A ¨* X 

In [21]:
df_recursos_new
resultado = A_new.dot(vector_costos)

dataframe_resultado = pd.DataFrame(resultado)

dataframe_resultado

,0
0,1307969.0
1,1369888.0
2,1370090.0
3,1452950.0
4,1438681.0
5,1531656.0
6,1557838.0
7,1552567.0
8,1700447.0
9,1413942.0


Logrando asi tener nuestro vector costos de productos para los nuevos productos

g) ¿Podr´ıa obtenerse la informaci´on de los precios que obtuvo en d) solo con la informaci´on de 40 productos
antiguos?

Si tenemos 40 productos, tendriamos una matrix 40x50, y un vector costos de productos 50x1, de esta manera para que se pudiera obtener informacion de los precios con solo 40 productos antiguos, nuestro vector costos de prodcutos deberia reducirse en 10, quedando 40x1 para que asi podamos obtener obtener una resolucion de sistemas lineales legal, es decir, tendriamos nuestra matriz 40x50, un vector incognita 50x1, y nuestro vector costos de productos 40x1, de esta manera seria posible obtener informacion a partir de 40 productos antiguos, pero esto llevaria que tuvieramos mayor cantidad de incognitas que de ecuaciones lineales, lo que significa que tendremos infinitas soluciones a nuestro sistema lo cual podriamos obtener informacion, pero no seria una informacion util si tuvieramos la misma cantidad de incognitas que de ecuaciones lineales como este establecido al principio. 

h) ¿Ser´ıa ´util de alg´un modo que los archivos Recursos.csv y Costos prod.csv tuviesen la informaci´on de
m´as productos antiguos? ¿Por qu´e?

Todo dependera de los nuevos productos que vayamos agregando, la idea es que la cantidad de productos que vayamos agregando no sea linealmente dependientes de los 50 productos ya existen, de esta manera no estariamos agregando columnas y filas basuras a nuestro sistema de ecuaciones lineales que al final y al cabo no estaran aportando niguna informacion util a nuestro sistema, en cambio  si agregamos prodcutos cuyas combinaciones lineales son efectivamente linealmente independientes, nuestro sistema se vera enriquecido, es decir si agregamos 10 productos mas al sistema, tambien se tendran que agregar sus 10 costos de produccion respectivos, pero tendriamos cierto problema dado que tendriamos una mayor cantidad de ecuaciones que de incognitas lo cual convierte a nuestro sistema en sobredeterminado pero para que este sistema tenga solucion, el vector de costos de produccion tendra que vivir dentro del espacio columna de la matriz A, lo cual no siempre ocurrira, respondiendo a la pregunta finalmente no seria tan util que los archivos tuvieran mas informacion de productos antiguos, dado que tenemos solo 50 incognitas de las materias primas. 